In [6]:
import deep_image_matching as dim

params = {
    "dir": "./assets/example_cyprus",
    "pipeline": "sift+lightglue",
    "strategy": "bruteforce",
    "quality": "high",
    "tiling": "none",
    "camera_options": "./assets/example_cyprus/cameras.yaml",
    "openmvg": None,
    "verbose": True,
}
config = dim.Config(params)

2024-05-14 11:27:49 | [WARNING ] assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high already exists, but the '--force' option is used. Deleting the folder.


In [7]:
config.extractor["n_features"] = 10000
config.extractor["nOctaveLayers"] = 4
config.extractor["contrastThreshold"] = 0.001
config.extractor["edgeThreshold"] = 10
config.extractor["sigma"] = 1.6

Run feature extraction and matching


In [8]:
matcher = dim.ImageMatcher(config)
feature_path, match_path = matcher.run()

2024-05-14 11:27:49 | [DEBUG   ] Matching options: Quality: HIGH - Tiling: NONE
2024-05-14 11:27:49 | [DEBUG   ] Saving directory: assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high
2024-05-14 11:27:49 | [DEBUG   ] Running inference on device cuda
2024-05-14 11:27:49 | [DEBUG   ] Matching options: Tiling: NONE
2024-05-14 11:27:49 | [DEBUG   ] Saving directory: assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high
2024-05-14 11:27:49 | [DEBUG   ] Running inference on device cuda


Loaded LightGlue model
2024-05-14 11:27:49 | [INFO    ] Running image matching with the following configuration:
2024-05-14 11:27:49 | [INFO    ]   Image folder: assets/example_cyprus/images
2024-05-14 11:27:49 | [INFO    ]   Output folder: assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high
2024-05-14 11:27:49 | [INFO    ]   Number of images: 10
2024-05-14 11:27:49 | [INFO    ]   Matching strategy: bruteforce
2024-05-14 11:27:49 | [INFO    ]   Image quality: HIGH
2024-05-14 11:27:49 | [INFO    ]   Tile selection: NONE
2024-05-14 11:27:49 | [INFO    ]   Feature extraction method: sift
2024-05-14 11:27:49 | [INFO    ]   Matching method: lightglue
2024-05-14 11:27:49 | [INFO    ]   Geometric verification: PYDEGENSAC
2024-05-14 11:27:49 | [INFO    ]   CUDA available: True
2024-05-14 11:27:49 | [DEBUG   ] Bruteforce matching, generating pairs ..
2024-05-14 11:27:49 | [INFO    ] Number of pairs: 45
2024-05-14 11:27:49 | [INFO    ] Found 45 pairs.
2024-05-14 11:27:49 | [INFO

  0%|          | 0/10 [00:00<?, ?it/s]

2024-05-14 11:27:49 | [DEBUG   ] Extracted 10000 keypoints


 10%|█         | 1/10 [00:00<00:07,  1.28it/s]

...

2024-05-14 11:28:11 | [DEBUG   ] Matching image pair: DSC_6473.JPG - DSC_6475.JPG
2024-05-14 11:28:11 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high/features.h5
2024-05-14 11:28:11 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high/features.h5
2024-05-14 11:28:11 | [DEBUG   ] Tile selection was NONE. Matching full images...
2024-05-14 11:28:12 | [DEBUG   ] [Timer] | [MatcherBase match] load h5 features=0.045, match full images=0.099, save to h5=0.001, Total execution=0.145
2024-05-14 11:28:12 | [DEBUG   ] PYDEGENSAC found 485 inliers (90.15%)
2024-05-14 11:28:12 | [DEBUG   ] Estiamted Fundamental matrix: 
[[-1.35637031e-04  3.63958502e-03 -2.96316305e+00]
 [-3.79253106e-03 -2.89724541e-04 -3.13919713e-01]
 [ 2.84519381e+00  1.00000000e+00 -3.97468070e+02]]
2024-05-14 11:28:12 | [DEBUG   ] [Timer] | [MatcherBase match] Geom. verification=0.119, save to h5=0.001, Total execution=

 98%|█████████▊| 44/45 [00:14<00:00,  2.72it/s]

2024-05-14 11:28:12 | [DEBUG   ] Matching image pair: DSC_6474.JPG - DSC_6475.JPG
2024-05-14 11:28:12 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high/features.h5
2024-05-14 11:28:12 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high/features.h5
2024-05-14 11:28:12 | [DEBUG   ] Tile selection was NONE. Matching full images...
2024-05-14 11:28:12 | [DEBUG   ] [Timer] | [MatcherBase match] load h5 features=0.046, match full images=0.097, save to h5=0.001, Total execution=0.144
2024-05-14 11:28:12 | [DEBUG   ] PYDEGENSAC found 571 inliers (92.54%)
2024-05-14 11:28:12 | [DEBUG   ] Estiamted Fundamental matrix: 
[[-1.18433431e-05 -2.59213151e-05  4.99765984e-02]
 [ 3.17414482e-04 -7.47175591e-06 -1.11974502e+00]
 [-8.97617745e-02  1.00000000e+00  4.46180112e+00]]
2024-05-14 11:28:12 | [DEBUG   ] [Timer] | [MatcherBase match] Geom. verification=0.107, save to h5=0.001, Total execution=

100%|██████████| 45/45 [00:15<00:00,  2.94it/s]

2024-05-14 11:28:12 | [INFO    ] [Timer] | [matching] generate_pairs=99.756, extract_features=7.707, Match pair=0.378, Total execution=122.800
2024-05-14 11:28:12 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000


Export matches to COLMAP format


In [9]:
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=config.general["output_dir"] / "database.db",
    camera_config_path=config.general["camera_options"],
)

2024-05-14 11:28:12 | [WARNING ] Database path assets/example_cyprus/results_sift+lightglue_bruteforce_quality_high/database.db already exists - deleting it
2024-05-14 11:28:12 | [WARNING ] Was not possible to load the first image to initialize cam0
2024-05-14 11:28:12 | [WARNING ] Was not possible to load the first image to initialize cam1


100%|██████████| 10/10 [00:00<00:00, 740.08it/s]
45it [00:00, 5396.38it/s]             
44it [00:00, 6592.23it/s]             


Run reconstruction with pycolmap


In [10]:
model = dim.reconstruction.pycolmap_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    sfm_dir=config.general["output_dir"],
    image_dir=config.general["image_dir"],
    refine_intrinsics=False,
)

2024-05-14 11:28:12 | [INFO    ] Running 3D reconstruction...


I20240514 11:28:12.810319 466242 misc.cc:198] 
Loading database
I20240514 11:28:12.812754 466242 database_cache.cc:54] Loading cameras...
I20240514 11:28:12.812825 466242 database_cache.cc:64]  1 in 0.000s
I20240514 11:28:12.812850 466242 database_cache.cc:72] Loading matches...
I20240514 11:28:12.813066 466242 database_cache.cc:78]  44 in 0.000s
I20240514 11:28:12.813083 466242 database_cache.cc:94] Loading images...
I20240514 11:28:12.815517 466242 database_cache.cc:143]  10 in 0.002s (connected 10)
I20240514 11:28:12.815572 466242 database_cache.cc:154] Building correspondence graph...
I20240514 11:28:12.818686 466242 database_cache.cc:190]  in 0.003s (ignored 0)
I20240514 11:28:12.818742 466242 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240514 11:28:12.819898 466242 misc.cc:198] 
Finding good initial image pair
I20240514 11:28:12.902141 466242 misc.cc:198] 
Initializing with image pair #5 and #9
I20240514 11:28:12.902958 466242 misc.cc:198] 
Global bundle adjustment
I20240514 11

2024-05-14 11:28:14 | [INFO    ] Reconstructed 1 model(s).
2024-05-14 11:28:14 | [INFO    ] Largest model is #0 with 10 images.
2024-05-14 11:28:14 | [INFO    ] Exporting model...


I20240514 11:28:14.770922 466242 timer.cc:91] Elapsed time: 0.033 [minutes]
